# Trabalho prático 2 - Estruturas Criptográficas

#### Autores: Ariana Lousada (PG47034), Cláudio Moreira (PG47844)
#### Grupo  12

## Kyber 

A última técnica a implementar tem como objetivo a implementação de um KEM IND-CPA e um PKE IND-CCA, do protótipo **Crystalis Kyber**.
<br/>
De seguida, são apresentadas em duas seções (PKE-IND-CPA e PKE-IND-CCA) os resultados da implementação de cada ténica. Esta resolução foi construída com base no documento *kyber.pdf* mais recente do site disponibilizado pela equipa docente. 

### KEM-IND-CPA

Esta versão permite obter uma segurança do tipo IND-CPA (Chosen Plaintext Attacks).
</br>
Em primeiro lugar foram desenvolvidas funções auxiliares de implementação de aritmética, encode, decode entre outras:

- ***parse***: recebe como *input* um conjunto de *bytes* e retorna o polinómio correspondente a esse conjunto;

- ***XOF***: corresponde a uma função do tipo *extendable output function*, utilizando *SHAKE-128*;

- ***PRF***: corresponde a uma função do tipo *pseudorandom function*, utilizando o *SHAKE-256*;

- ***G***: função de *hash* construída com base no *SHA3-512*;

- ***encode***: recebe um polinómio como argumento e retorna um *byte array* correspondente ao polinómio;

- ***decode***: função inversa da ***encode*** - recebe um *byte array* e retorna o polinómio correspondente;

- ***compress***: comprime um polinómio e retorna os *bytes* correspondentes;

- ***decompress***: função inversa da ***compress***. Descomprime *bytes* e retorna o polinómio correspondente;

- ***transposta***: calcula a transposta de uma matriz.

As funções principais centram-se na geração de chaves, cifragem da mensagem passada como parâmetro e posterior decifragem do texto cifrado, obtendo deste modo, a mensagem original:

- ***gerar\_chaves***: com recurso às funções anteriormente apresentadas, ocorre a geração das chaves pública e secreta. A primeira é essencial para a cifragem da mensagem e a segunda para a decifragem do criptograma;

- ***cifragem***: tem como objetivo principal a cifragem de uma mensagem. Desta forma, recebe como parâmetros a chave pública, a mensagem e *coins* (*bytes* aleatórios) e dá como *output* o texto cifrado;

- ***decifragem***: tem como objetivo decifrar um criptograma, obtendo como resultado o texto limpo correspondente. Recebe como argumentos a chave secreta e o criptograma.

In [17]:
import os
import random as rn
from sympy import ntt
from cryptography.hazmat.primitives import hashes
import numpy
from sympy import intt
import gzip
import struct

In [18]:
# constantes Kyber 
n = 256
q = 343576577
k = 2
n1 = 3
n2 = 2
du, dv = 10, 4

# criação dos anéis 
_Z.<w>  = ZZ[]
R.<w>   = QuotientRing(_Z ,_Z.ideal(w^n - 1))

_Q.<w>  = GF(q)[]
Rq.<w>  = QuotientRing(_Q , _Q.ideal(w^n + 1))


# tamanho necessário para o decompress
def tamanho(stringB, numberS):
    count = 10
    auxCount = 1
    i = 0
    while i < len(stringB):
        if numberS == auxCount:
            i = i + 10
            while (i < len(stringB)) and (stringB[i] != 31 or stringB[i + 1] != 139 or stringB[i + 2] != 8 or stringB[i + 3] != 0  ):
                count = count + 1
                i = i + 1
            auxCount = auxCount + 1

        i = i + 1
        if (i + 10) < len(stringB) and (stringB[i] == 31 and stringB[i + 1] == 139 and stringB[i + 2] == 8 and stringB[i + 3] == 0 ):
            auxCount = auxCount + 1
        if auxCount > numberS:
            break
    return count

# input: conjunto de bytes; output: polinómio do conjunto de bytes inserido;
def parse(str_bytes):
    result = []
    for i in str_bytes:
        result.append(i)
    return Rq(result)

# XOF, com o SHAKE-128
def XOF(p,i,j):
    digest = hashes.Hash(hashes.SHAKE128(int(32)))
    digest.update(p)
    digest.update(bytes(i))
    digest.update(bytes(j))
    r = digest.finalize()
    return r

# pseudorandom function com SHAKE-256
def PRF(s,b):
    digest = hashes.Hash(hashes.SHAKE256(int(32)))
    digest.update(s)
    digest.update(bytes(b))
    r = digest.finalize()
    return r

# função de hash com SHA3-512;
def G(d):
    digest = hashes.Hash(hashes.SHA512())
    digest.update(bytes(d))
    r = digest.finalize()
    return r

# input: polinómio; output: byte array;
def encode(poly):
    byt=b''
    aux=1
    countX=0
    for j in poly:
        if(j>255):
            aux=2
        if (j > 65025):
            aux = 3
        if (j > 16581375):
            aux = 4
        if (j > 4228250625):
            aux = 5
        byt = byt+ int((_Z(j))).to_bytes( aux, 'big')
        byt = byt +"/-n-/".encode()
        countX =countX +1
    return byt

# input: byte array; output: polinómio correspondente ao byte array do input;
def decode(byt):
    listaCoef = []
    byteAux = b''
    listAux = []
    desc=0
    while desc <byt.__len__():
        if byt[desc] == 47 and byt[desc+1]==45 and byt[desc+2]==110 and byt[desc+3]==45 and byt[desc+4] == 47 :
            desc = desc+4
            listaCoef.append(int.from_bytes(byteAux, 'big'))
            byteAux = b''
        else:
            byteAux = byteAux + bytearray([int(_Z(byt[desc]))])
        desc = desc+1
    return listaCoef

# comprime um polinómio em bytes
def compress(polinomio):
    polinomioB= encode(polinomio)
    compress = gzip.compress(polinomioB)
    return compress

# descomprime os bytes e transfora-os num polinómio
def decompress(compress):
    unpack = gzip.decompress(compress)
    return Rq(decode(unpack))

# calcula a transposta de uma matriz
def transposta(matrix):
    zipped_rows = zip(*matrix)
    transpose_matrix = [list(row) for row in zipped_rows]
    return transpose_matrix

# geração do par de chaves (publickey, secretkey)
def gerar_chaves():
    d = bytearray(os.urandom(32))
    p = G(d)[:32]
    teta = G(d)[-32:]
    N = 0
    A = [[ 0 for x in range(k-1)] for y in range(k-1)]
    for i in range(0,k-1):
        for j in range(0,k-1):
            A[i][j] =parse(XOF(p,i,j))
    s = []
    for i in range(0,k-1):
        s.append(parse(PRF(teta,N)))
        N = N + 1
    e = []
    for i in range(0,k-1):
        e.append(parse(PRF(teta,N)))
        N = N + 1
    s1 = Rq(T.ntt(s[0]))
    e1 = Rq(T.ntt(e[0]))
    t = A[0][0].lift() * s1.lift() + e1.lift()
    pk = encode(t) + p
    sk = encode(s1)
    return pk, sk
  
# cifra uma mensagem m
def cifragem(pk, m, coins):
    N = 0
    t2 = pk[:len(pk)-32]
    t= decode(t2)
    p = pk[-32:]
    A = [[ 0 for x in range(k-1)] for y in range(k-1)]
    for i in range(0,k-1):
        for j in range(0,k-1):
            A[i][j] =parse(XOF(p,i,j))
    AT = transposta(A)
    r = []
    for i in range(0,k-1):
        r.append(parse((PRF(bytearray(r),bytearray([N])))))
        N = N + 1
    e1 = []
    for i in range(0,k-1):
        e1.append(parse(PRF(bytearray(r[i]),bytearray([N]))))
        N = N + 1
    e2 = parse(PRF(bytearray(r[0].list()),N))
    r1= Rq(T.ntt(r[0]))
    u= Rq(T.ntt_inv(A[0][0].lift()*r1.lift()))+e1[0]
    v = Rq(T.ntt(Rq(t).lift() * r1.lift())) + e2 + decompress(m)
    c1 = compress(u)
    c2 = compress(v)
    c = c1 + c2
    return c
    
# decifragem do criptograma
def decifragem(sk, ciphertext):
    u = decompress(ciphertext[:tamanho(ciphertext,1)])
    v = decompress(ciphertext[-tamanho(ciphertext,2):])
    s1 = Rq(decode(sk))
    m = compress(v.lift() - (T.ntt_inv(s1.lift()*Rq(T.ntt(u)).lift())))
    return m
    

De seguida são apresentados os resultados obtidos com recurso às funções anteriores. Neste caso, estamos a considerar uma mensagem fixa.
</br>
Note-se que houve uma dificuldade acrescida nesta implementação pelo que, deparamos-nos com alguns erros não identificados que impossibilitam o correto funcionamento do programa.

In [19]:
pk, sk = gerar_chaves()

m = Rq([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
            0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
            1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1])

print("Mensagem = ", compress(m), "\n")
coins = bytearray(os.urandom(32))
ciphertext = cifragem(pk, compress(m),coins)
print("Texto cifrado = ", ciphertext, "\n")
texto = decifragem(sk, ciphertext)
print("Texto limpo = ", texto)

Mensagem =  b'\x1f\x8b\x08\x00\xf3\xc8mb\x02\xffc\xd4\xd7\xcd\xd3\xd5g\x00\x93\x8cD\x90\x0cD\x88 \x8bc\x92\xa4\x9a\xc6@\xa2\x0bq\xe9b\xa0\xd8w\xe4\x99\xc9H\x96\x1f\x19(\x88\x17\x06\x12}DL<R+m\x10\xef\x1eF\x12\xd3\xd5(9J\x8e\x92\xa4\x92\x00\xe2{\x855\x00\x06\x00\x00' 

Texto cifrado =  b'\x1f\x8b\x08\x00\xf3\xc8mb\x02\xff5UyT\xd4U\x14\x1ef\x06fa\x16`\x06\xf5\x90,\xa1\x89\xec)\xc3"t\x12\xd9\x86MBb \xc0\x8e\xa4Dp\x0c\xa23%\x18\x11\x8bMA R\x08X"d*\xc6\x91E\x8cP@\x16#\x01\x01\tA\x8aS\x90$\xa6@\x94\x88,\x02\x99\x9d\xf3\xbb\x1f\xff}\xe7\xde\xf7\xbew\xefw\x97\xc7\x12\x06\xa4}co\x9bhk\xcf\xd2\xf3\xb3?MH\xf7Y\xf6\xaf\xb0i\x86\xbe"$\xec\xce*%\xa4\xef(\x08d\x10{\xc7\xbdz2\xe9\x1c\x9eo`\x10\xa7p\xf5\x00\xf9\xae\x9dO#\xcbx\x8a\x01X\x95\xcf\xf6\x93\xef\xe3\xccq\x98\xd2\x97\x94\x0cr\x1b\x90\x93E\xd6n\xf0>\x83\x8aT\xff\x90\xc5`b\xac\x1d\xaf\x1c\xffw\x9e8]\x85Md\x12\x07g\xa4#\xd2\xc5\xa9`B\x92\xe8C\xcb\x0c\xdaz\xe2\x0bz\xefl\xc6\x0cB\xcf\x99,\x00k\xf9Nor*6=O\xa4\x99\x92\xef\xf1\xcej\x8f\x8c|%jW\xb07\xeb

## Kyber CPAPKE

Começou-se com a implementação do algoritmo que permite uma segurança do tipo IND-CPA, isto é, contra Chosen Plaintext Attacks.

Sendo assim, começou-se com a implementação de funções auxiliares:

- ***parse***: Através de um conjunto de bytes, devolve um polinómio.

- ***XOF***: Corresponde a uma função do tipo extendable output function.

- ***PRF***: Corresponde a uma função do tipo pseudorandom function.

- ***G***: Realiza o *hash* através do SHA3-512;

- ***encode***: Devolve um array de bytes através de um polinómio.

- ***decode***: Possui um comportamento contrário ao encode.

- ***compress***: Realiza a compressão de um polinómio.

- ***decompress***: Função oposta ao ***compress***.

- ***transposta***: Calcula a transposta de uma matriz.

De seguida, realizou-se a construção das funções principais:

- ***gerar\_chaves***: Realiza a geração de chaves privadas e públicas.

- ***cifragem***: Realiza a cifragem da mensagem através da chave pública.

- ***decifragem***: Decifra um criptograma, devolvendo um texto limpo.

No entanto, houveram dificuldades na construção da função *ntt* e *ntt_inversa* o que impossibilitou o bom funcionamento destas funções.

In [1]:
import os
import random as rn
from sympy import ntt
from cryptography.hazmat.primitives import hashes
import numpy
from sympy import intt
import gzip
import struct

In [2]:
n = 256
q = 343576577
T = NTT(n,q)
k = 2
n1 = 3
n2 = 2
du, dv = 10, 4

_Z.<w>  = ZZ[]
R.<w>   = QuotientRing(_Z ,_Z.ideal(w^n - 1))

_Q.<w>  = GF(q)[]
Rq.<w>  = QuotientRing(_Q , _Q.ideal(w^n + 1))

def ntt():
    return

def ntt_inv():
    return

def tamanho(stringB, numberS):
    contador = 10
    auxContador = 1
    i = 0
    while i < len(stringB):
        if numberS == auxContador:
            i = i + 10
            while (i < len(stringB)) and (stringB[i] != 31 or stringB[i + 1] != 139 or stringB[i + 2] != 8 or stringB[i + 3] != 0  ):
                countador = countador + 1
                i = i + 1
            auxContador = auxContador + 1

        i = i + 1
        if (i + 10) < len(stringB) and (stringB[i] == 31 and stringB[i + 1] == 139 and stringB[i + 2] == 8 and stringB[i + 3] == 0 ):
            auxContador = auxContador + 1
        if auxContador > numberS:
            break
    return countador

def parse(str_bytes):
    result = []
    for i in str_bytes:
        result.append(i)
    return Rq(result)

def XOF(p,i,j):
    digest = hashes.Hash(hashes.SHAKE128(int(32)))
    digest.update(p)
    digest.update(bytes(i))
    digest.update(bytes(j))
    r = digest.finalize()
    return r

def PRF(s,b):
    digest = hashes.Hash(hashes.SHAKE256(int(32)))
    digest.update(s)
    digest.update(bytes(b))
    r = digest.finalize()
    return r

def G(d):
    digest = hashes.Hash(hashes.SHA512())
    digest.update(bytes(d))
    r = digest.finalize()
    return r

def encode(poly):
    byte=b''
    aux=1
    countadorX=0
    for j in poly:
        if(j>255):
            aux=2
        if (j > 65025):
            aux = 3
        if (j > 16581375):
            aux = 4
        if (j > 4228250625):
            aux = 5
        byte = byte+ int((_Z(j))).to_bytes( aux, 'big')
        byte = byte +"/-n-/".encode()
        countadorX =countadorX +1
    return byt

def decode(byt):
    listaCoeficiente = []
    byteAux = b''
    desc=0
    while desc <byt.__len__():
        if byt[desc] == 47 and byt[desc+1]==45 and byt[desc+2]==110 and byt[desc+3]==45 and byt[desc+4] == 47 :
            desc = desc+4
            listaCoeficiente.append(int.from_bytes(byteAux, 'big'))
            byteAux = b''
        else:
            byteAux = byteAux + bytearray([int(_Z(byt[desc]))])
        desc = desc+1
    return listaCoeficiente

def compress(polinomio):
    polinomioB= encode(polinomio)
    compress = gzip.compress(polinomioB)
    return compress

def decompress(compress):
    unpack = gzip.decompress(compress)
    return Rq(decode(unpack))

def transposta(matrix):
    zipped_rows = zip(*matrix)
    transpose_matrix = [list(row) for row in zipped_rows]
    return transpose_matrix

def gerar_chaves():
    d = bytearray(os.urandom(32))
    p = G(d)[:32]
    teta = G(d)[-32:]
    N = 0
    A = [[ 0 for x in range(k-1)] for y in range(k-1)]
    for i in range(0,k-1):
        for j in range(0,k-1):
            A[i][j] =parse(XOF(p,i,j))
    s = []
    for i in range(0,k-1):
        s.append(parse(PRF(teta,N)))
        N = N + 1
    e = []
    for i in range(0,k-1):
        e.append(parse(PRF(teta,N)))
        N = N + 1
    s1 = Rq(T.ntt(s[0]))
    e1 = Rq(T.ntt(e[0]))
    t = A[0][0].lift() * s1.lift() + e1.lift()
    pk = encode(t) + p
    sk = encode(s1)
    return pk, sk
  
def cifragem(pk, m, coins):
    N = 0
    t2 = pk[:len(pk)-32]
    t= decode(t2)
    p = pk[-32:]
    A = [[ 0 for x in range(k-1)] for y in range(k-1)]
    for i in range(0,k-1):
        for j in range(0,k-1):
            A[i][j] =parse(XOF(p,i,j))
    AT = transposta(A)
    r = []
    for i in range(0,k-1):
        r.append(parse((PRF(bytearray(r),bytearray([N])))))
        N = N + 1
    e1 = []
    for i in range(0,k-1):
        e1.append(parse(PRF(bytearray(r[i]),bytearray([N]))))
        N = N + 1
    e2 = parse(PRF(bytearray(r[0].list()),N))
    r1= Rq(T.ntt(r[0]))
    u= Rq(T.ntt_inv(A[0][0].lift()*r1.lift()))+e1[0]
    v = Rq(T.ntt(Rq(t).lift() * r1.lift())) + e2 + decompress(m)
    c1 = compress(u)
    c2 = compress(v)
    c = c1 + c2
    return c
    
def decifragem(sk, ciphertext):
    u = decompress(ciphertext[:tamanho(ciphertext,1)])
    v = decompress(ciphertext[-tamanho(ciphertext,2):])
    s1 = Rq(decode(sk))
    m = compress(v.lift() - (T.ntt_inv(s1.lift()*Rq(T.ntt(u)).lift())))
    return m
    

NameError: name 'NTT' is not defined

## Resultados obtidos

In [3]:
pk, sk = gerar_chaves()

m = Rq([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
            0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
            1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
            1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1])
print("Mensagem = ", compress(m), "\n")
coins = bytearray(os.urandom(32))
texto_cifrado = cifragem(pk, compress(m),coins)
print("Texto cifrado = ", texto_cifrado, "\n")
texto = decifragem(sk, texto_cifrado)
print("Texto limpo = ", texto)


NameError: name 'gerar_chaves' is not defined